In [1]:
! pip install wandb opencv-python-headless==4.1.2.30 albumentations torch-summary timm==0.5.4 einops joblib icecream  -qq -U

     |████████████████████████████████| 1.7 MB 8.2 MB/s 
     |████████████████████████████████| 21.8 MB 1.3 MB/s 
     |████████████████████████████████| 102 kB 74.2 MB/s 
     |████████████████████████████████| 431 kB 55.9 MB/s 
     |████████████████████████████████| 97 kB 8.2 MB/s 
     |████████████████████████████████| 142 kB 59.3 MB/s 
     |████████████████████████████████| 180 kB 71.4 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 


In [2]:
import warnings
from glob import glob
import pathlib
from pathlib import Path
from torchsummary import summary
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pprint import pprint
import urllib.request
import csv
import numpy as np
from einops import rearrange, reduce, repeat
from torch.cuda import amp
from tqdm import tqdm
import wandb
import time
import copy
from collections import defaultdict
from sklearn.metrics import mean_squared_error
import joblib
import gc
import os
from icecream import ic
from sklearn.model_selection import train_test_split
import gc
import cv2
import copy
import time
import random
from PIL import Image

# For data manipulation
import numpy as np
import pandas as pd
from einops import rearrange, repeat
from scipy import stats

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict
import matplotlib.pyplot as plt

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold

import timm

import json

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# For colored terminal text
from colorama import Fore, Back, Style
c_ = Fore.CYAN
sr_ = Style.RESET_ALL

warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"


# ENV

In [50]:

# ENV = 'COLAB'
ENV = 'KAGGLE'
# ENV = 'SYSTEM'

# Option for Mixed Precision
# FP16 = True
FP16 = False


CONFIG = dict(
    seed=42,
    backbone=None,
    embedder=None,
    train_batch_size=8,
    valid_batch_size=64,
    img_size=384,
    num_epochs=5,
    early_stopping=True,
    early_stopping_step=10,
    learning_rate=1e-4,
    scheduler='CosineAnnealingLR',
    min_lr=1e-6,
    T_max=100,
    num_classes=25,
    weight_decay=1e-6,
    device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    competition='lg',
    _wandb_kernel='deb'
)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [4]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)


set_seed(CONFIG['seed'])


In [48]:
import wandb


run = wandb.init(project="lg", entity="jiwon7258", config = CONFIG, job_type = 'inf', id='1u5wdz84', resume='must' )
dataset = wandb.run.use_artifact(
    'jiwon7258/lg/lg:v1', type='dataset')

    
# Download the artifact's contents
dataset_dir = dataset.download()
dataset_dir = Path(dataset_dir)


wandb: Downloading large artifact lg:v1, 9188.81MB. 121115 files... Done. 0:0:0


# Augmentation

In [28]:
data_transforms = {
    "train": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05,
                           rotate_limit=15, p=0.5),
        A.RGBShift(r_shift_limit=15, g_shift_limit=15,
                   b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Normalize(),
        ToTensorV2(),
        ], p=1.),

    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(),
        ToTensorV2()], p=1.)
}


# Dataset Dataloader

In [51]:
TEST_PATH =  dataset_dir / 'test'

In [52]:
test_csv = sorted(glob(str(TEST_PATH / '*/*.csv')))
test_jpg = sorted(glob(str(TEST_PATH / '*/*.jpg')))

In [53]:
# # TTA
# class TestDataset(Dataset):
#     def __init__ (self, N, test_imgs, transforms = None):
#         self.N = N
#         self.test_imgs = test_imgs
#         self.transforms = transforms


#     def __len__ (self):
#         return len(self.test_imgs)

#     def __getitem__(self, index):
#         img_path = self.test_imgs[index]
#         img_code = (Path(img_path)).parent.stem
#         img = cv2.imread(img_path)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
#         imgs = [self.transforms(image=img)['image'] for _ in range(self.N)]
#         imgs = np.concatenate(imgs, axis = 0)
#         imgs = rearrange(imgs, '(new1 bs) h w -> new1 bs h w', new1=self.N)

#         return imgs, img_code

# testDataset = TestDataset(8, test_jpg, transforms = data_transforms['train'])
# testDataloader = DataLoader(
#     testDataset, batch_size=CONFIG['valid_batch_size'], shuffle=False)


In [54]:
class TestDataset(Dataset):
    def __init__(self, test_imgs, transforms=None):
        self.test_imgs = test_imgs
        self.transforms = transforms

    def __len__(self):
        return len(self.test_imgs)

    def __getitem__(self, index):
        img_path = self.test_imgs[index]
        img_code = (Path(img_path)).parent.stem
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        if (self.transforms) :
            img = self.transforms(image=img)['image']

        return img, img_code


testDataset = TestDataset(test_jpg, transforms=data_transforms['valid'])
testDataloader = DataLoader(
    testDataset, batch_size=CONFIG['valid_batch_size'], shuffle=False)


In [55]:
# for imgs, img_code in testDataset :
#     fig, axes = plt.subplots(1,8, figsize = (40,20))
#     for i in range(8):
#         img = imgs[i, :, :, :]
#         axes[i].imshow(img.reshape(384,384,3))
#     break

In [56]:
for img, img_code in testDataloader:
    print(type(img))
    print(img.shape)
    print(img_code)
    print(type(img_code))
    break


<class 'torch.Tensor'>
torch.Size([64, 3, 384, 384])
('10000', '10001', '10002', '10003', '10004', '10005', '10006', '10007', '10008', '10009', '10010', '10011', '10012', '10013', '10014', '10015', '10016', '10017', '10018', '10019', '10020', '10021', '10022', '10023', '10024', '10025', '10026', '10028', '10029', '10030', '10031', '10032', '10033', '10034', '10035', '10036', '10038', '10039', '10040', '10041', '10042', '10044', '10046', '10047', '10048', '10049', '10050', '10051', '10052', '10053', '10054', '10055', '10056', '10057', '10058', '10059', '10060', '10061', '10062', '10064', '10065', '10066', '10067', '10068')
<class 'tuple'>


# Key Value Dict

In [57]:
wandb.restore('class_dict', run_path='jiwon7258/lg/1lkvc6n0', root='./')
class_dict = joblib.load('class_dict')

# MODELS

## EfficientNet V2 M

In [58]:
class Effnet(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super().__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.n_features = 1280
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        output = self.fc(features)
        return output


effnet = Effnet('tf_efficientnetv2_m',None, pretrained=False)
effent = effnet.to(CONFIG['device'])


In [59]:
wandb.restore('epoch3_Loss0.0760.bin', 'jiwon7258/lg/koxrlhhf', root='./')
effnet.load_state_dict(torch.load('epoch3_Loss0.0760.bin',
                      map_location=CONFIG['device']))


<All keys matched successfully>

## Swin

In [60]:
class Swin(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super(Swin, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.n_features = self.backbone.head.in_features
        self.backbone.reset_classifier(0)
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        output = self.fc(features)
        return output


swin = Swin('swin_base_patch4_window12_384',
            None, pretrained=False)
swin.to(CONFIG['device'])


Swin(
  (backbone): SwinTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (layers): Sequential(
      (0): BasicLayer(
        dim=128, input_resolution=(96, 96), depth=2
        (blocks): ModuleList(
          (0): SwinTransformerBlock(
            (norm1): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (attn): WindowAttention(
              (qkv): Linear(in_features=128, out_features=384, bias=True)
              (attn_drop): Dropout(p=0.0, inplace=False)
              (proj): Linear(in_features=128, out_features=128, bias=True)
              (proj_drop): Dropout(p=0.0, inplace=False)
              (softmax): Softmax(dim=-1)
            )
            (drop_path): Identity()
            (norm2): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
            (mlp): Mlp(
              (

In [61]:
wandb.restore('epoch3_Loss0.1026.bin', 'jiwon7258/lg/2mzc3731', root='./')
swin.load_state_dict(torch.load('epoch3_Loss0.1026.bin',
                      map_location=CONFIG['device']))


<All keys matched successfully>

## DeiT

In [62]:
class Deit(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super(Deit, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.n_features = 768
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        if isinstance(features, tuple):
          features = features[0]
        output = self.fc(features)
        return output


deit = Deit('deit_base_distilled_patch16_384',
            None, pretrained=False)
deit.to(CONFIG['device'])


Deit(
  (backbone): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU()
          (drop1): Dropout(p=0.0, inplace=False)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop2): Dropout(p=0.0, inplace=False

In [63]:
wandb.restore('epoch13_Loss0.0081.bin', 'jiwon7258/lg/2g7lsew4', root='./')
deit.load_state_dict(torch.load(
    'epoch13_Loss0.0081.bin', map_location=CONFIG['device']))


<All keys matched successfully>

## EfficientNet B4 NS

In [64]:
class Noisy(nn.Module):
    def __init__(self, backbone, embedder, pretrained=True):
        super(Noisy, self).__init__()
        self.backbone = timm.create_model(backbone, pretrained=pretrained)
        self.backbone.reset_classifier(0)
        self.n_features = 1792
        self.fc = nn.Linear(self.n_features, CONFIG['num_classes'])

    def forward(self, images):
        # features = (bs, embedding_size)
        features = self.backbone(images)
        # outputs  = (bs, num_classes)
        # if isinstance(features, tuple):
        #   features = features[0]
        output = self.fc(features)
        return output


noisy = Noisy('tf_efficientnet_b4_ns', None, pretrained=False)
noisy.to(CONFIG['device'])


Noisy(
  (backbone): EfficientNet(
    (conv_stem): Conv2dSame(3, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (act1): SiLU(inplace=True)
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
          (bn1): BatchNorm2d(48, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act1): SiLU(inplace=True)
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Conv2d(48, 24, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn2): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_runni

In [65]:
wandb.restore('epoch50_Loss0.0127.bin', 'jiwon7258/lg/zut3hsab', root='./')
noisy.load_state_dict(torch.load(
    'epoch50_Loss0.0127.bin', map_location=CONFIG['device']))


<All keys matched successfully>

In [66]:
models = []
models.append(effnet)
models.append(swin)
models.append(deit)
models.append(noisy)

In [67]:
# # TTA
# img_code_list = []
# outputs = []
# with torch.no_grad():
#     model.eval()

#     bar = tqdm(enumerate(testDataloader), total=len(testDataloader))

#     for step, (imgs, img_codes) in bar:
#         # imgs : (bs, N, C, H, W), torch.Tensor
#         # img_codes (bs, N), list(str)
#         imgs = imgs.to(CONFIG['device'])
#         imgs = rearrange(imgs, 'bs N C H W -> (bs N) C H W')
#         logits = model(imgs)
#         # logits : ( (bs * N), num_classes)
#         logits = logits.detach().cpu()
#         logits = rearrange(
#             logits, '(bs N) num_classes -> bs N num_classes', bs=CONFIG['valid_batch_size'])
#         output = np.argmax(torch.softmax(logits, dim=-1), axis=-1)
#         output = np.array(output)
#         # (bs, N)
#         output = stats.mode(output, axis=1)[0].reshape(-1)
#         # (bs)
#         output = [class_dict[i] for i in output]

#         print(img_codes)
#         print(output)

#         img_code_list.extend(img_codes)
#         outputs.extend(output)


In [68]:
# no TTA
with torch.no_grad():

    for model in models:
        model.eval()

    img_code_list = []
    outputs = []
    bar = tqdm(enumerate(testDataloader), total=len(testDataloader))

    for step, (img, img_code) in bar:
        # img : (bs, C, H, W), torch.Tensor
        # img_code : (bs)
        img = img.to(device)
        batch_size = img.shape[0]

        probs_list=[]
        for model in models:
            logits = model(img)
            logits = logits.cpu()
            # (bs, num_classes)
            # 모델들마다 logit들의 값의 크기가 다를 수 있으므로, softmax를 이용해 정규화하는 작업이 필요하다
            prob = torch.softmax(logits, dim = -1)
            probs_list.append(prob)
        
        probs_list = torch.stack(probs_list)
        # (num_model, bs, num_clases)
        probs = torch.sum(probs_list, dim = 0)
        # (bs, num_classes)

        output = np.argmax(torch.softmax(probs, dim=-1), axis=-1) # torch.Tensor, (bs,)
        output = np.array(output)
        output = [class_dict[i] for i in output]

        img_code_list.extend(img_code)
        outputs.extend(output)

        assert len(img_code_list) == len(outputs)




100%|██████████| 812/812 [1:00:34<00:00,  4.48s/it]


In [69]:
result = pd.DataFrame()
result['image'] = img_code_list
result['label'] = outputs
SUBMIT_FILE_NAME = 'submit_effnetv2m_swin_deit_noisy.csv'
result.to_csv(SUBMIT_FILE_NAME,index= False, index_label=False)

In [70]:
wandb.save(SUBMIT_FILE_NAME)


['/content/wandb/run-20220117_111755-1u5wdz84/files/submit_effnetv2m_swin_deit_noisy.csv']